# **Setup**
## Imports

In [1]:
# Auto-reload edited scripts
%load_ext autoreload
%autoreload 2

# Logging
from ngram_acquire.pipeline.logger import setup_logger

# Ngram pivot functions
from pathlib import Path
from ngram_pivot.config import PivotConfig
from ngram_pivot.pivot import pivot_database
from ngram_pivot.stream import stream_year_ngrams, stream_year_range, get_ngram_stats

# Utilities
from utilities.peek import *

## Set up logging to file

In [2]:
setup_logger(
    db_path="/vast/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/5grams_pivoted.db",
    console=False,
    rotate=True,
    max_bytes=100_000_000,
    backup_count=5,
    force=True
)

PosixPath('/vast/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/ngram_download_20251007_163949.log')

# **Pivot Multigrams to Create Yearly Indices**

In [3]:
# Configure and run the pivot operation
config = PivotConfig(
    source_db_path=Path("/vast/edk202/NLP_corpora/Google_Books/20200217/eng/1gram_files/1grams_processed.db"),
    target_db_path=Path("/vast/edk202/NLP_corpora/Google_Books/20200217/eng/1gram_files/1grams_pivoted.db"),
    write_batch_size=500_000,
    log_interval=500_000,
    validate=True
)

# Run the pivot
pivot_database(config)

N-GRAM DATABASE PIVOT
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Start Time: 2025-10-07 16:39:50

Configuration
════════════════════════════════════════════════════════════════════════════════════════════════════
Source DB:            /vast/edk202/NLP_corpora/Google_Books/20200217/eng/1gram_files/1grams_processed.db
Target DB:            /vast/edk202/NLP_corpora/Google_Books/20200217/eng/1gram_files/1grams_pivoted.db
Source profile:       read:packed24
Target profile:       write:packed24
Write batch size:     500,000
Validation enabled:   True

Progress
════════════════════════════════════════════════════════════════════════════════════════════════════


Pivoting [970,500,000 written, batch 0.3s]:  |                   | 13.5M [22:57<00:00, 9.80kngram/s]



Pivot Complete
════════════════════════════════════════════════════════════════════════════════════════════════════
Source n-grams:              13,499,384
Year records found:          970,620,875
Target records written:      970,620,875
Errors:                      0
Throughput:                  704,291 records/sec
Total runtime:               0:22:58

End Time:                    2025-10-07 17:02:49
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━


:qq# **Inspect the Procesed Database**
## `db_head`: Print the first _N_ key–value pairs

In [4]:
db_path = "/vast/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/5grams_processed.db"

db_head(db_path, key_format="utf-8", value_format="packed", n=5)

First 5 key-value pairs:
────────────────────────────────────────────────────────────────────────────────────────────────────
[ 1] Key:   <UNK> <UNK> <UNK> <UNK> aaa
     Value: [166 records] ... +156 earlier, (2010, 471, 403), (2011, 413, 327), (2012, 362, 299)
            (2013, 267, 224), (2014, 309, 256), (2015, 220, 190), (2016, 232, 204)
            (2017, 194, 184), (2018, 132, 107), (2019, 139, 115)

[ 2] Key:   <UNK> <UNK> <UNK> <UNK> aac
     Value: [122 records] ... +112 earlier, (2010, 124, 103), (2011, 131, 97), (2012, 138, 114)
            (2013, 178, 87), (2014, 176, 134), (2015, 58, 53), (2016, 96, 86), (2017, 121, 74)
            (2018, 115, 74), (2019, 29, 27)

[ 3] Key:   <UNK> <UNK> <UNK> <UNK> aachen
     Value: [143 records] ... +133 earlier, (2010, 216, 152), (2011, 235, 173), (2012, 1900, 1363)
            (2013, 1181, 944), (2014, 223, 169), (2015, 132, 120), (2016, 207, 162)
            (2017, 257, 141), (2018, 192, 140), (2019, 144, 121)

[ 4] Key:   <UNK> <U

## `db_peek`: Print _N_ key-value pairs starting at the specified key

In [5]:
db_path = "/vast/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/5grams_processed.db"

db_peek(db_path, start_key=b"quick brown <UNK> <UNK> <UNK>", key_format="utf-8", value_format="packed", n=5)


5 key-value pairs starting from 717569636b2062726f776e203c554e4b3e203c554e4b3e203c554e4b3e:
────────────────────────────────────────────────────────────────────────────────────────────────────
[ 1] Key:   quick brown <UNK> <UNK> <UNK>
     Value: [21 records] ... +11 earlier, (2006, 4, 3), (2007, 1, 1), (2008, 13, 3), (2009, 2, 2)
            (2010, 5, 4), (2011, 2, 2), (2012, 9, 7), (2013, 5, 3), (2014, 2, 1), (2016, 1, 1)

[ 2] Key:   quick brown eye <UNK> <UNK>
     Value: [156 records] ... +146 earlier, (2010, 5, 5), (2011, 6, 6), (2012, 10, 10), (2013, 19, 19)
            (2014, 18, 18), (2015, 22, 22), (2016, 18, 18), (2017, 26, 26), (2018, 99, 99)
            (2019, 16, 16)

[ 3] Key:   quick brown eye <UNK> butler
     Value: [10 records] (1866, 23, 23), (1867, 2, 2), (1869, 2, 2), (1870, 2, 2), (1871, 4, 4), (1875, 3, 3)
            (1891, 1, 1), (1892, 1, 1), (1903, 1, 1), (1908, 1, 1)

[ 4] Key:   quick brown eye take <UNK>
     Value: [36 records] ... +26 earlier, (2007, 1,

## `db_peek_prefix`: Print key-value pairs containing the specified prefix

In [9]:
db_path = "/vast/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/5grams_processed.db"

db_peek_prefix(db_path, prefix=b"<UNK> united state <UNK> <UNK>", key_format="utf-8", value_format="summary", n=1)

1 key-value pairs with prefix 3c554e4b3e20756e69746564207374617465203c554e4b3e203c554e4b3e:
────────────────────────────────────────────────────────────────────────────────────────────────────
[ 1] Key:   <UNK> united state <UNK> <UNK>
     Value: Total: 163,384,713 occurrences in 96,901,345 volumes (1472-2019, 384 years)

